In [21]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import requests 
import time
from fake_useragent import UserAgent

In [22]:
scraper = cloudscraper.create_scraper()
url = 'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&foot_min=40&offer_type=flat&only_foot=2&p=2&region=-1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1&sort=creation_date_desc'
response = requests.get(url)
tree = BeautifulSoup(response.content, 'html.parser')
tree.title.text.strip()

'Купить 1, 2, 3, 4, 5-комнатную квартиру, многокомнатную квартиру, квартиру-студию в Москве и Московской области - 137 300 объявлений'

In [23]:
scraper = cloudscraper.create_scraper()
def get_href(p):
    time.sleep(2)
    url = 'https://www.cian.ru/cat.php?deal_type=sale&engine_version=2&foot_min=40&offer_type=flat&only_foot=2&p={}&region=-1&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room9=1&sort=creation_date_desc'.format(p)
    response = scraper.get(url)
    tree = BeautifulSoup(response.text, 'html.parser')
    books = tree.find_all('article', {"data-name": "CardComponent"})
    infa = [ ]
    for book in books:
        infa.append({'Ссылка' : book.find('a').get('href')})
    return infa

def get_info(b):
    time.sleep(3)
    url2 = infa[b]['Ссылка']
    responsed = scraper.get(url2)
    tree = BeautifulSoup(responsed.content, 'html.parser')
    try:
       full = {'Ссылка': infa[b]['Ссылка'],
            'Комнаты': tree.find('h1', {"class": "a10a3f92e9--title--vlZwT"}).find(string=True), 
            'Цена' : (tree.find('div', {"data-testid": "price-amount"})).find(string=True), 
            'Цена за метр' : (tree.find('div', {'data-name' : "OfferFactItem"})).find_all(string=True)[1],
           'Метро' : (tree.find('a', {"class": "a10a3f92e9--underground_link--VnUVj"})).find(string=True),
           'Время до метро(мин)' : (tree.find('span', {"class": "a10a3f92e9--underground_time--YvrcI"})).find(string=True),
           'Тип' : (tree.find('div', {"data-name": 'OfferSummaryInfoLayout'})).find_all(string=True),
           'Этаж' : (tree.find_all('span', {"style":"letter-spacing:-0.2px"}))}
    except:
        full = {'Ссылка': infa[b]['Ссылка'],
            'Комнаты': 'NaN', 
            'Цена' : 'NaN', 
            'Цена за метр' : 'NaN',
           'Метро' : 'NaN',
           'Время до метро(мин)' : 'NaN',
           'Тип' : 'NaN',
           'Этаж' : 'NaN'}
        
    return full

In [24]:
infa = []
for p in range(1,45):
    infa.extend(get_href(p))
for b in range(len(infa)):
    infa[b] = (get_info(b))
    
import pandas as pd

df = pd.DataFrame(infa)
print(df.shape)
df

(1231, 8)


,Ссылка,Комнаты,Цена,Цена за метр,Метро,Время до метро(мин),Тип,Этаж
0,https://www.cian.ru/sale/flat/301871490/,"Продается 2-комн. квартира, 61,2 м²",27 300 000 ₽,446 078 ₽/м²,Полежаевская,17 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[2-комн. квартира, 61,2 м²], [61,2 м²], [31 м..."
1,https://mytishchi.cian.ru/sale/flat/301871465/,"Продается 3-комн. квартира, 71,5 м²",15 400 000 ₽,215 385 ₽/м²,Медведково,9 мин.,"[О квартире, Тип жилья, Новостройка, Общая пло...","[[3-комн. квартира, 71,5 м²], [71,5 м²], [42,4..."
2,https://khimki.cian.ru/sale/flat/301871438/,"Продается 2-комн. квартира, 44,7 м²",8 050 000 ₽,180 089 ₽/м²,Левобережная,9 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[2-комн. квартира, 44,7 м²], [44,7 м²], [30,2..."
3,https://www.cian.ru/sale/flat/301871419/,"Продается 2-комн. квартира, 51 м²",19 490 000 ₽,382 157 ₽/м²,Багратионовская,7 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[2-комн. квартира, 51 м²], [51 м²], [8 м²], [..."
4,https://solnechnogorsk.cian.ru/sale/flat/30187...,"Продается 2-комн. квартира, 52,7 м²",7 000 000 ₽,132 827 ₽/м²,Зеленоград — Крюково,6 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[2-комн. квартира, 52,7 м²], [52,7 м²], [27,6..."
...,...,...,...,...,...,...,...,...
1226,https://www.cian.ru/sale/flat/301810887/,"Продается 4-комн. квартира, 95,7 м²",26 300 000 ₽,274 817 ₽/м²,Кузьминки,14 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[4-комн. квартира, 95,7 м²], [95,7 м²], [61 м..."
1227,https://ramenskoye.cian.ru/sale/flat/301810877/,"Продается Студия, 30,6 м²",5 500 000 ₽,179 739 ₽/м²,Фабричная,5 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[Студия, 30,6 м²], [30,6 м²], [26,8 м²], [9 и..."
1228,https://reutov.cian.ru/sale/flat/301810832/,"Продается Студия, 29,7 м²",6 600 000 ₽,222 222 ₽/м²,Новокосино,15 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[Студия, 29,7 м²], [29,7 м²], [28,3 м²], [16 ..."
1229,https://www.cian.ru/sale/flat/301810831/,"Продается 2-комн. квартира, 56,2 м²",22 350 000 ₽,397 687 ₽/м²,Белорусская,9 мин.,"[О квартире, Тип жилья, Вторичка, Общая площад...","[[2-комн. квартира, 56,2 м²], [56,2 м²], [37,2..."


In [78]:
x = list()
y = list()
u = list()
f = list()
meta = list()
feta = list()
for i in range(len(df)):
   try:
      x.append('.'.join((str(df.loc[i,['Комнаты']]).split(','))[1:3]).split(' ')[1])
   except:
      x.append('NaN')
   try:
      if (((str(df.loc[i,['Комнаты']])).split(' '))[5])[0] == 'А' or (((str(df.loc[i,['Комнаты']])).split(' '))[5])[0] == 'С':
          y.append(1)
      elif (((str(df.loc[i,['Комнаты']])).split(' '))[5])[0] == 'М':
          y.append(6)
      else:
          y.append((((str(df.loc[i,['Комнаты']])).split(' '))[5])[0])
   except:
      y.append('NaN')
   df.loc[i,['Цена за метр']] = (((str(df['Цена за метр'][i])).replace(' ','')).replace('₽/м²',''))
   df.loc[i,['Цена']] = (((str(df['Цена'][i])).replace('\xa0', '')).replace('₽',''))
   df.loc[i,['Время до метро(мин)']] = ((str(df['Время до метро(мин)'][i])).replace(' мин.', ''))
   u.append(((df['Тип'][i][2]).split(' '))[0])
   if int(len(df['Тип'][i][2].split(' '))) == 2:
      f.append('Апартаменты')
   else:
      f.append('Квартира')
   for l in range(len(df['Этаж'][i])):
    try:         
       n = df['Этаж'][i][l].text.split(' ')
       if 'из' in n:
          meta.append(n[2])
          feta.append(n[0])
          break
    except:
          meta.append('NaN')
          feta.append('NaN')
          break
df['Площадь(м2)'] = x
df['Комнаты'] = y  
df['Максимальный этаж'] = meta 
df.drop('Этаж', axis=1, inplace=True)
df.drop('Тип', axis=1, inplace=True)
df['Этаж'] = feta     
df['Тип строения'] = f        
df['Тип жилья'] = u       

In [93]:
print(len(df))
for i in range(len(df)):
    if df['Цена'][i] == 'NaN':
        df.drop([i],inplace=True)
print(len(df)) 

1227


KeyError: 389

In [94]:
df.to_csv('Cian.csv', encoding='utf-8')